In [ ]:
# Tuning visualization for CNN-LSTM (signals)
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Configure display
sns.set(context="notebook", style="whitegrid")

# Locate results CSV (prefer ./results, fallback ./saved_models)
results_path = "./results/tuning_signals_results.csv"
if not os.path.isfile(results_path):
    alt = "./saved_models/tuning_signals_results.csv"
    if os.path.isfile(alt):
        results_path = alt
    else:
        raise FileNotFoundError("Could not find tuning_signals_results.csv under ./results or ./saved_models")

# Load
df = pd.read_csv(results_path)
# Coerce numeric (in case some are strings)
for col in ["learning_rate","batch_size","hidden_size","num_layers","dropout","val_acc","val_loss"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

print(f"Loaded {len(df)} trials from: {results_path}")
df.head()

In [ ]:
# Quick overview
print(df.describe(include="all"))
# Top-10 by validation accuracy
best10 = df.sort_values("val_acc", ascending=False).head(10)
best10


In [ ]:
# Scatter overview: val_acc vs learning_rate, encoding other hparams
plt.figure(figsize=(8,5))
sns.scatterplot(data=df, x="learning_rate", y="val_acc",
                hue="hidden_size", style="num_layers", size="batch_size", sizes=(30,120))
plt.title("Validation accuracy by learning rate")
plt.tight_layout()
os.makedirs("./results", exist_ok=True)
plt.savefig("./results/tune_scatter_lr.png", dpi=200)
plt.show()


In [ ]:
# Heatmaps: batch_size vs hidden_size, faceted by dropout (cols) and num_layers (rows) per learning_rate
import numpy as np
for lr in sorted(df.learning_rate.unique()):
    sub = df[df.learning_rate == lr]
    g = sns.FacetGrid(sub, col="dropout", row="num_layers", margin_titles=True, height=3)
    def draw_heat(data, color=None):
        pv = data.pivot_table(index="batch_size", columns="hidden_size", values="val_acc", aggfunc=np.max)
        sns.heatmap(pv, annot=True, fmt=".2f", cmap="viridis")
    g.map_dataframe(draw_heat)
    g.fig.suptitle(f"Val Acc heatmaps (lr={lr})", y=1.03)
    out = f"./results/tune_heatmap_lr{lr}.png"
    g.savefig(out, dpi=200)
    plt.show()


In [ ]:
# Parallel coordinates to compare all hyperparameters vs val_acc
from pandas.plotting import parallel_coordinates

plot_cols = ["learning_rate","batch_size","hidden_size","num_layers","dropout","val_acc"]
# Normalize columns 0..1 for plotting
norm = df[plot_cols].copy()
for c in plot_cols:
    mn, mx = norm[c].min(), norm[c].max()
    if mx > mn:
        norm[c] = (norm[c] - mn) / (mx - mn)
    else:
        norm[c] = 0.0
# Create quartile bins for coloring
bins = pd.qcut(df["val_acc"], q=4, labels=["Q1","Q2","Q3","Q4"])
norm["acc_quartile"] = bins
plt.figure(figsize=(10,5))
parallel_coordinates(norm[plot_cols + ["acc_quartile"]], "acc_quartile", colormap="viridis", alpha=0.5)
plt.title("Parallel coordinates of hyperparameters vs val_acc (normalized)")
plt.tight_layout()
plt.savefig("./results/tune_parallel.png", dpi=200)
plt.show()


In [ ]:
# Bar chart of top-k configurations by val_acc
k = 10
bestk = df.sort_values("val_acc", ascending=False).head(k).copy()
bestk["cfg"] = bestk.apply(lambda r: f"lr={r.learning_rate}, bs={int(r.batch_size)}, h={int(r.hidden_size)}, L={int(r.num_layers)}, d={r.dropout}", axis=1)
plt.figure(figsize=(10,5))
sns.barplot(data=bestk, x="val_acc", y="cfg", orient="h", palette="viridis")
plt.xlabel("Validation Accuracy (%)")
plt.ylabel("Configuration")
plt.title(f"Top {k} configurations by validation accuracy")
plt.tight_layout()
plt.savefig("./results/tune_topk.png", dpi=200)
plt.show()
